In [1]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import copy
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from ipywidgets import interact

# **0. VGG 모델 개요**

- 이미지 분류 CNN 모델들 중에 하나
- 몇 개의 층(layer)으로 구성되어 있는지에 따라, 16개 층으로 구성되어 있으면 VGG16,19개 층으로 구성되어 있으면 VGG19라고 불림
- VGGNet 연구의 핵심은 **네트워크의 깊이**가 성능에 어떤 영향을 미치는지를 확인하고자 한 것
> 컨볼루션 필터 커널의 사이즈를 가장 작은 3x3
- https://minjoos.tistory.com/6

# **1. VGG19 모델 불러오기**

In [2]:
model = models.vgg19(pretrained = True) # 이미 학습된 모델

c:\users\doroc\appdata\local\programs\python\python37\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
c:\users\doroc\appdata\local\programs\python\python37\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
from torchsummary import summary # 모델 구조 요약

In [4]:
summary(model,(3,224,224),batch_size = 1,device = 'cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [1, 64, 224, 224]           1,792
              ReLU-2          [1, 64, 224, 224]               0
            Conv2d-3          [1, 64, 224, 224]          36,928
              ReLU-4          [1, 64, 224, 224]               0
         MaxPool2d-5          [1, 64, 112, 112]               0
            Conv2d-6         [1, 128, 112, 112]          73,856
              ReLU-7         [1, 128, 112, 112]               0
            Conv2d-8         [1, 128, 112, 112]         147,584
              ReLU-9         [1, 128, 112, 112]               0
        MaxPool2d-10           [1, 128, 56, 56]               0
           Conv2d-11           [1, 256, 56, 56]         295,168
             ReLU-12           [1, 256, 56, 56]               0
           Conv2d-13           [1, 256, 56, 56]         590,080
             ReLU-14           [1, 256,

# **2. 데이터에 맞게 모델의 Head 부분 수정하기**

In [5]:
### Head 부분을 다음과 같이 수정

model.avgpool = nn.AdaptiveAvgPool2d(output_size = (1,1)) # 각 구역의 평균값 출력
model.classifier = nn.Sequential(
    nn.Flatten(), # 평탄화
    nn.Linear(512,256), # 512 -> 256
    nn.ReLU(), # 활성화 함수(비선형성 부여)
    nn.Dropout(0.1), # 과적합 방지
    nn.Linear(256,7), # 256 -> 7(7개의 감정으로 분류되니)
    nn.Sigmoid() # 활성화 함수, 이진 분류
)

In [6]:
### 모델 생성 함수

def build_vgg19_based_model(device_name = 'cpu'):
    device = torch.device(device_name)
    model = models.vgg19(pretrained = True) # 이미 학습된 vgg19 모델 불러오기
    # 일반 NN Layer(FC Layer)
    model.avgpool = nn.AdaptiveAvgPool2d(output_size = (1,1)) # 각 구역의 평균값 출력
    model.classifier = nn.Sequential(
        nn.Flatten(), # 평탄화
        nn.Linear(512,256), # 512 -> 256
        nn.ReLU(), # 활성화 함수
        nn.Dropout(0.1), # 과적합 방지
        nn.Linear(256,7), # 256 -> 7(7개의 감정으로 분류되니)
        nn.Softmax() # 활성화 함수(각 클래스에 속할 확률 추정)
    )
    
    return model.to(device)

In [7]:
model = build_vgg19_based_model(device_name = 'cpu') # 모델 객체 생성

In [8]:
summary(model,(3,224,224),batch_size = 1,device = 'cpu') # 모델 구조 요약

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [1, 64, 224, 224]           1,792
              ReLU-2          [1, 64, 224, 224]               0
            Conv2d-3          [1, 64, 224, 224]          36,928
              ReLU-4          [1, 64, 224, 224]               0
         MaxPool2d-5          [1, 64, 112, 112]               0
            Conv2d-6         [1, 128, 112, 112]          73,856
              ReLU-7         [1, 128, 112, 112]               0
            Conv2d-8         [1, 128, 112, 112]         147,584
              ReLU-9         [1, 128, 112, 112]               0
        MaxPool2d-10           [1, 128, 56, 56]               0
           Conv2d-11           [1, 256, 56, 56]         295,168
             ReLU-12           [1, 256, 56, 56]               0
           Conv2d-13           [1, 256, 56, 56]         590,080
             ReLU-14           [1, 256,

c:\users\doroc\appdata\local\programs\python\python37\lib\site-packages\torch\nn\modules\container.py:139: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


# **3. 손실함수 정의**

In [9]:
loss_func = nn.CrossEntropyLoss(reduction = 'mean')

# **4. 옵티마이져 정의**

In [10]:
optimizer = torch.optim.SGD(model.parameters(),lr = 1e-3,momentum = 0.9)